In [1]:
import scanpy as sc
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import numpy as np
from matplotlib.colors import TwoSlopeNorm
import math
from matplotlib.patches import Circle

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-s7gc6wyn because the default path (/home/stereonote/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [ ]:
sample = 'sample_tmp'
input_dir = '/data/work/TLS_shells'
data_path = f'{input_dir}/{sample}/{sample}.metadata.csv'
#Input metadata containing TLS label, XY coordinates, cell annotations, and other relevant information.
data = pd.read_csv(data_path,index_col=0)

TLS_namelist = data['TLS_name'].unique().tolist()
TLS_namelist.remove("other")

In [ ]:
#The geometric center of all B-cell coordinates is defined as the TLS (tertiary lymphoid structure) center. TLS without B-cell annotation are excluded from analysis.
center_points = {}
for TLS in TLS_namelist:
    TLS_file = data[data['TLS_name']==TLS]
    if TLS_file['cell2loc_anno'].isin(['B cell']).any():
        Bcell_file  = TLS_file[TLS_file['cell2loc_anno'].isin(['B cell'])]
        center_x = Bcell_file['x'].mean()
        center_y = Bcell_file['y'].mean()
        center_points[TLS] = (center_x, center_y)
    else:
        print(TLS)

In [ ]:
def calculate_dynamic_radius(tls_data,center,max_size=2400):
    points = tls_data[['x', 'y']].values
    longest_side=max(np.sqrt(np.sum((points - np.array(center))**2, axis=1)))
    base_radius = 1000
    return max(math.ceil(base_radius / 200) * 200, max_size)
def add_expansion_zones(data, center_points, step=200):
    result_data = data.copy()
    for TLSname, center in center_points.items():
        if center is None:
            continue    
        points = result_data[['x', 'y']].values
        distances = np.sqrt(np.sum((points - np.array(center))**2, axis=1))
        max_radius = calculate_dynamic_radius(result_data[result_data['TLS_name'] == TLSname],center)
        radii = list(range(step, max_radius + step, step))
        
        result_data[f'{TLSname}_expansion'] = 'other'
        for i in range(len(radii)-1, -1, -1):
            radius = radii[i]
            if i==0:
                mask = (distances <= radius)
                zone_label = f'{radius/2}'
            else:
                prev_radius = radii[i-1]
                mask = (distances > prev_radius) & (distances <= radius)
                zone_label = f'{radius/2}'
            result_data.loc[mask, f'{TLSname}_expansion'] = zone_label
    return result_data

In [7]:
expanded_data = add_expansion_zones(data, center_points)

In [ ]:
output_path = f'/data/work/TLS_shells/result/{sample}'
os.makedirs(output_path, exist_ok = True)
expanded_data.to_csv(f'{output_path}/{sample}.expansion.only1200um.csv',index=True)

In [ ]:
#Fig 5A
for TLSname, center in center_points.items():
    center_x, center_y = center
    draw_data = expanded_data[['x','y','TLS_name',f'{TLSname}_expansion']]
    conditions = [
            (draw_data[f'{TLSname}_expansion'] != 'other') & (draw_data['TLS_name'] == TLSname),
            (draw_data[f'{TLSname}_expansion'] != 'other'),
            True
        ]
    colors = ['#FFA500', '#ADD8E6', '#D3D3D3']
    draw_data['color'] = np.select(conditions, colors)
    
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.scatter(draw_data['x'], draw_data['y'], c=draw_data['color'], alpha=0.8, s=0.5, edgecolors='none')
        
    ax.scatter(center_x, center_y, c='red', marker='x', s=1, linewidths=2, zorder=5)
    radiusList = draw_data[f'{TLSname}_expansion'].unique().tolist()
    radiusList.remove("other")
    for radius in radiusList:
        circle = Circle((center_x, center_y), float(radius)*2,
                           fill=False, linestyle='-', 
                           linewidth=0.2, alpha=0.8,
                           edgecolor='black', zorder=3)
        ax.add_patch(circle)
    ax.set_aspect('equal')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.axis('off')
    plt.title(f'{TLSname}')
    plt.savefig(f'{output_path}/{TLSname}.expansion.only1200um.pdf', dpi=800)
    plt.close()

/tmp/ipykernel_662/4129288225.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  draw_data['color'] = np.select(conditions, colors)
/tmp/ipykernel_662/4129288225.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  draw_data['color'] = np.select(conditions, colors)
/tmp/ipykernel_662/4129288225.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [12]:
print('ggg')

ggg
